# Azure OpenAI Assistants API - Streaming RAG Chat FastAPI

Notebook contains logic for testing an Azure OpenAI Assistants API deployment designed to answer questions from a RAG store (Azure AI Search Index). This notebook submits requests to either a local or cloud-hosted FastAPI endpoint which contains logic for creating threads, submitting messages, calling tools, and streaming chat responses. 

### Import required packages

In [1]:
import requests
import json
from IPython.display import display, HTML, Markdown, clear_output
import os

import threading
from dotenv import load_dotenv

load_dotenv(override=True)

uri = 'http://127.0.0.1:8000'

### Streaming display helper function

In [2]:
# Function to process and display the streamed response
def process_streamed_response(response):
    buffer = ''
    try:
        for line in response.iter_lines(decode_unicode=True):
            if line:
                buffer += line + '\n'
                clear_output(wait=True)
                display(Markdown(buffer))
               
            
        # Ensure the final content is displayed
        clear_output(wait=True)
        display(Markdown(buffer))
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        response.close()

### Workflow - Step #1: Create a Thread

Following request creates a new Assistants API-managed thread

In [3]:
response = requests.post(f"{uri}/create_thread")
thread_id = response.json()

### Workflow - Step #2: Submit User Query as New Message

New message is added to the thread which is subsequently run. The response is then streamed and displayed here inline.

In [4]:
url = f"{uri}/run_assistant"

data={
    'thread_id': thread_id, 
    'message': 'How do I turn on the Bike?'
}

# Send the POST request with stream=True
response = requests.post(url, json=data, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Start a separate thread to process the response
    thread = threading.Thread(target=process_streamed_response, args=(response,))
    thread.start()
else:
    print(f"Error: {response.status_code}")
    print(response.text)

<i>Retrieving Documents...</i> {'keywords': 'turn on Peloton Bike', 'document_count': 3} <br><br>
To turn on the Peloton Bike, follow these steps:
1. Hold the power button for two seconds to power on the Bike.
2. The Peloton Bike will sleep after about 20 minutes of inactivity. To wake the Bike, press the power button.
3. To power down, hold the power button for two seconds and select "Shut Down" when prompted [Peloton Bike Manual US_CA_page_25.pdf].


### Workflow - Step #3: Continue Conversation

Follow up with another question that builds on the initial question. Note that context is maintained via the Assistants API state management.

In [5]:
url = f"{uri}/run_assistant"

data={
    'thread_id': thread_id, 
    'message': 'How about the Row?'
}

# Send the POST request with stream=True
response = requests.post(url, json=data, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Start a separate thread to process the response
    thread = threading.Thread(target=process_streamed_response, args=(response,))
    thread.start()
else:
    print(f"Error: {response.status_code}")
    print(response.text)

<i>Retrieving Documents...</i> {'keywords': 'turn on Peloton Row', 'document_count': 3} <br><br>
To turn on the Peloton Row, follow these steps:
1. Locate the power button on the back of the touchscreen.
2. Press and hold the power button to turn on the Peloton Row [Peloton_Row_Manual_US_page_6.pdf].


In [6]:
url = f"{uri}/run_assistant"

data={
    'thread_id': thread_id, 
    'message': 'How do I maintain all of this equipment?'
}

# Send the POST request with stream=True
response = requests.post(url, json=data, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Start a separate thread to process the response
    thread = threading.Thread(target=process_streamed_response, args=(response,))
    thread.start()
else:
    print(f"Error: {response.status_code}")
    print(response.text)


<i>Retrieving Documents...</i> {'keywords': 'Peloton Bike maintenance', 'document_count': 3} <br><br>
<i>Retrieving Documents...</i> {'keywords': 'Peloton Row maintenance', 'document_count': 3} <br><br>
### Peloton Bike Maintenance
**After Your First Few Rides:**
- **Pedals:** Use the included 15 mm wrench to tighten pedals [Peloton Bike Manual US_CA_page_30.pdf].
**Every 3 to 5 Rides:**
- **Cleaning:** Clean the Peloton Bike thoroughly using a damp cloth and a mild household cleaner diluted in water. Clean the touchscreen using an electronics wipe.
- **Cleats:** Use a 4mm Allen wrench to tighten any loose screws attaching your cleats to your cycling shoes.
- **Saddle:** Make sure the saddle is parallel to the ground. Then use the included 13mm wrench to tighten the seat-fixing nuts on both sides [Peloton Bike Manual US_CA_page_32.pdf].
**Every 15 to 20 Rides:**
- **Emergency Brake:** Press down on the resistance knob while pedaling slowly. The flywheel should come to a complete stop immediately. If it does not, stop using the Bike immediately and contact Peloton Support.
- **Frame:** Check the entire frame for signs of wear and damage, and tighten any loose bolts [Peloton Bike Manual US_CA_page_32.pdf].
**Annually:**
- **Pedals:** Replace pedals. For replacement instructions, see support.onepeloton.com [Peloton Bike Manual US_CA_page_33.pdf].
### Peloton Row Maintenance
**Before Every Class:**
- Check the area around and underneath the Peloton Row for potential obstructions.
- Inspect the Peloton Row and do not use it if it appears damaged or inoperable [Peloton_Row_Manual_US_page_16.pdf].
**Monthly:**
- Check the power cord for damage.
- Check the entire Row for wear and damage, especially the handle strap and foot straps.
- Clean the area underneath the Row.
- Remove the front cover of the Upright Wall Anchor and verify that the tether is secured to the mounting bracket and not frayed, and that the bolts are secured to the wall.
- Clean all visible surfaces of the Row thoroughly using a damp cloth and a mild household cleaner diluted in water. Do not use harsh cleaning products containing bleach or ammonia [Peloton_Row_Manual_US_page_16.pdf].
